In [86]:
import pandas as pd
import warnings
from utils.data import contains_text

warnings.filterwarnings("ignore")
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [87]:
voters_df = pd.read_csv("~/Desktop/Mali_sign_targets.csv")
voters_df.fillna(" ", inplace=True)

In [88]:
st_df = pd.read_csv("~/Desktop/school_streets.csv")
for c in ["street", "school"]:
    st_df[c] = [x.upper() for x in st_df[c]]
street_school_dict = dict(zip(st_df["street"], st_df["school"]))


streets = list(st_df.street)
teachers_df = voters_df.loc[[contains_text(a, streets) for a in voters_df.address]]
teachers_df["which_school"] = ""
for street in streets:
    teachers_df["which_school"] = [
        street_school_dict[street] if street in a else w
        for a, w in zip(teachers_df.address, teachers_df.which_school)
    ]

teachers_df.sort_values(by="which_school").to_clipboard()

### Merge Newton City Voter Data with L2 data

In [89]:
completed_lists = [
    "2A1",
    "2A2",
    "2a2.1",
    "2A3",
    "2A3.1",
    "2A3.3",
    "2A4",
    "5A1",
    "5A2",
    "5A3",
    "5A4",
    "5A5",
    "5A6",
    "5B1",
    "5B3.1",
    "5B3.2",
    "5B6",
    "6A1",
    "6B2",
    "6B3",
    "6B4",
    "6C4",
    "6C5",
    "6C1",
]

In [90]:
city_df = pd.read_csv("~/Desktop/voters_with_formid.csv", index_col=[0])
l2_df = pd.read_csv("/Volumes/T5_External/data/newton/2025_election/l2.csv")

filenames = [
    "~/Desktop/canvass_lists/canvass_set_6BC.csv",
    "~/Desktop/canvass_lists/canvass_set_5AB.csv",
    "~/Desktop/canvass_lists/canvass_set_2a.csv",
]
df_list = []
for fn in filenames:
    df_list.append(pd.read_csv(fn))

canvassed_df = pd.concat(df_list)

# append canvas info
city_df = city_df.merge(
    canvassed_df[["fullname", "canvass_list", "form_id"]],
    on=["fullname", "form_id"],
    how="left",
)

In [91]:
l2_df.columns.values

array(['LALVOTERID', 'Voters_Active', 'Voters_StateVoterID',
       'Voters_CountyVoterID', 'Voters_FirstName', 'Voters_MiddleName',
       'Voters_LastName', 'Voters_NameSuffix',
       'VoterTelephones_LandlineFormatted',
       'VoterTelephones_LandlineUnformatted',
       'VoterTelephones_LandlineConfidenceCode',
       'VoterTelephones_CellPhoneFormatted',
       'VoterTelephones_CellPhoneUnformatted',
       'VoterTelephones_CellConfidenceCode',
       'VoterTelephones_CellPhoneOnly', 'Residence_Addresses_AddressLine',
       'Residence_Addresses_ExtraAddressLine', 'Residence_Addresses_City',
       'Residence_Addresses_State', 'Residence_Addresses_Zip',
       'Residence_Addresses_ZipPlus4', 'Voters_SequenceZigZag',
       'Voters_SequenceOddEven', 'Residence_Addresses_CensusTract',
       'Residence_Addresses_CensusBlockGroup',
       'Residence_Addresses_CensusBlock', 'Residence_Addresses_Latitude',
       'Residence_Addresses_Longitude', 'Residence_Addresses_Density',
       

In [92]:
l2_df.columns.values
cols = [
    "Voters_FirstName",
    "Voters_StateVoterID",
    "Voters_SequenceOddEven",
    "Voters_Age",
    "Ethnic_Description",
    "VoterParties_Change_Changed_Party",
    "Vote_Frequency",
    "Voters_LastName",
    "Residence_Addresses_AddressLine",
    "VoterTelephones_CellPhoneFormatted",
    "VoterTelephones_CellConfidenceCode",
    "VoterTelephones_LandlineFormatted",
    "VoterTelephones_LandlineConfidenceCode",
    "Parties_Description",
    "Residence_HHParties_Description",
]
l2_subset_df = l2_df[cols]
l2_subset_df.columns = ["l2_" + x.lower() for x in l2_subset_df.columns]
l2_subset_df["l2_fullname"] = [
    f + " " + l
    for f, l in zip(l2_subset_df.l2_voters_firstname, l2_subset_df.l2_voters_lastname)
]
l2_subset_df.drop(columns=["l2_voters_firstname", "l2_voters_lastname"], inplace=True)

In [93]:
df = city_df.merge(
    l2_subset_df,
    left_on="voter_id_number",
    right_on="l2_voters_statevoterid",
    how="right",
)
df["house_contacted"] = [1 if c in completed_lists else 0 for c in df.canvass_list]

df.drop(
    columns=[
        "politics",
        "polling_location",
        "big_single_family",
        "educator",
        "finance",
        "muckity_muck",
        "helping",
        "multifamily",
        "men_over_60",
        "women",
    ],
    inplace=True,
)

In [94]:
# fix the rando streets
df["tmp"] = [
    x.split(" ", 1)[-1].upper().split(" APT ")[0]
    for x in df.l2_residence_addresses_addressline
]
df.fullstname.fillna(df.tmp, inplace=True)
df["tmp"] = [x.split(" ", 1)[0] for x in df.l2_residence_addresses_addressline]
df.number.fillna(df.tmp, inplace=True)
df["tmp"] = [x.upper() for x in df.l2_residence_addresses_addressline]
df.address.fillna(df.tmp, inplace=True)
df.fullname.fillna(df.l2_fullname, inplace=True)
df.age_on_election_day.fillna(df.l2_voters_age, inplace=True)

In [95]:
df.to_csv("~/Desktop/voter_city_with_l2.csv", index=False)

In [69]:
tmp_df = df[["ward", "house_contacted"]].groupby(["ward"]).count().reset_index()
tmp_df.columns = [
    "ward",
    "contacted",
]
tmp2_df = (
    df[["ward", "house_contacted"]]
    .loc[df.house_contacted == 1]
    .groupby(["ward"])
    .count()
    .reset_index()
)
tmp_df.merge(tmp2_df, on="ward", how="outer").to_clipboard()

In [35]:
l2_subset_df.merge(city_df, left_on="Voters_StateVoterID", right_on="voter_id_number")

,Voters_FirstName,Voters_StateVoterID,Voters_SequenceOddEven,Voters_Age,Ethnic_Description,VoterParties_Change_Changed_Party,Vote_Frequency,Voters_LastName,VoterTelephones_CellPhoneFormatted,VoterTelephones_LandlineFormatted,...,finance,muckity_muck,helping,multifamily,men_over_60,women,gender,lat,lon,form_id
0,Susan,05FSN0550001,362463817,75.0,German,NaN,7,Feldman,NaN,(617) 244-7484,...,0,0,0,0,0,1,female,42.329883,-71.224836,5341
1,Denise,01WDE0850000,362448147,75.0,German,NaN,8,Wernikoff,NaN,NaN,...,0,0,0,0,0,1,female,42.337697,-71.228518,32208
2,Lisa,01SLA0551000,362464766,74.0,Dutch (Netherlands),NaN,7,Sazer,(617) 448-3538,NaN,...,0,0,0,0,0,1,female,42.333439,-71.219653,53425
3,Fred,01CFD0651000,362486754,74.0,Polish,NaN,8,Chanowski,(617) 407-3112,(617) 965-2623,...,0,0,0,0,1,0,male,42.306124,-71.203353,8494
4,Janice,05AJE1750000,362454198,75.0,English/Welsh,NaN,8,Axelrod,(617) 610-4953,(617) 969-5398,...,0,0,0,0,0,1,female,42.351009,-71.247643,41247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6965,Nicholas,01BNS0180005,362466543,44.0,German,NaN,7,Brandt,(617) 894-2478,NaN,...,0,0,0,0,0,0,male,42.323953,-71.228394,54350
6966,Jessica,05ZJA2283000,362438889,42.0,Italian,NaN,7,Zerillo,NaN,NaN,...,0,0,0,1,0,1,female,42.356059,-71.213115,21184
6967,Eileen,02OEN0356003,362462014,69.0,Irish,NaN,6,Okeefe,(413) 563-1240,(617) 964-4611,...,0,0,0,0,0,1,female,42.322103,-71.218657,5287
6968,Yangyang,05ZYG1587000,362486530,38.0,Chinese,NaN,3,Zhao,(617) 909-5963,NaN,...,0,0,0,0,0,0,unknown,42.291647,-71.183775,8414


In [17]:
completed_lists = [
    "2A1",
    "2A2",
    "2a2.1",
    "2A3",
    "2A3.1",
    "2A3.3",
    "2A4",
    "5A1",
    "5A2",
    "5A3",
    "5A4",
    "5A5",
    "5A6",
    "5B1",
    "5B3.1",
    "5B3.2",
    "5B6",
    "6A1",
    "6B2",
    "6B3",
    "6B4",
    "6C4",
    "6C5",
    "6C1",
]

canvassed_df.loc[canvassed_df.canvass_list.isin(completed_lists)]

,form_id,address,apt,fullname,age_on_election_day,occupation,canvass_list
0,2479,15 BURR RD,,JEREMY EICHLER,51,JOURNALIST,6A1
1,2479,15 BURR RD,,KAREN NAIMER,52,UNKNOWN,6A1
2,2480,20 BURR RD,,ALEXANDER KOBE,30,STUDENT,6A1
3,2480,20 BURR RD,,JAMES KOBE,67,LAWYER,6A1
4,2480,20 BURR RD,,JAYNE BAILIN,70,LAWYER,6A1
...,...,...,...,...,...,...,...
876,53706,86 UPLAND RD,,ETHAN SEWALL,49,TEACHER,5B6
877,53706,86 UPLAND RD,,FREJA DOBREFF,29,,5B6
878,53706,86 UPLAND RD,,JAMES DOBREFF,60,UNKNOWN,5B6
879,53706,86 UPLAND RD,,RICHARD SEWALL,83,COACH/TEACHER,5B6


In [78]:
city_df

,addressid,wp_address_id,voter_id_number,fullname,first_name,number,address,fullstname,apt,party_affiliation,...,muckity_muck,helping,multifamily,men_over_60,women,gender,lat,lon,form_id,canvass_list
0,100858262,11100858262,11RSA2056000,SONJA RIOUX,SONJA,262,262 ADAMS ST,ADAMS ST,1,D,...,0,0,1,0,1,female,42.356797,-71.197938,110,NaN
1,100858322,11100858322,05RMN2248001,MARILYN ROGERS,MARILYN,322,322 ADAMS ST,ADAMS ST,,D,...,0,0,1,0,1,female,42.355366,-71.196026,118,NaN
2,100858290,11100858290,12FRT0354000,ROBERT FRANCHI,ROBERT,290,290 ADAMS ST,ADAMS ST,,U,...,0,0,0,1,0,male,42.356180,-71.196983,113,NaN
3,100858290,11100858290,08FJE1055000,JOANNE FRANCHI,JOANNE,290,290 ADAMS ST,ADAMS ST,,D,...,0,0,0,0,1,female,42.356180,-71.196983,113,NaN
4,100858316,11100858316,03YJE0556000,JEE YIP,JEE,316,316 ADAMS ST,ADAMS ST,,U,...,0,0,1,0,0,andy,42.355493,-71.196187,117,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47850,242059143,84242059143,09RAM0906000,ADAM ROTBERG,ADAM,143,143 WISWALL RD,WISWALL RD,,U,...,0,0,0,0,0,male,42.294626,-71.189181,84606,NaN
47851,2444595,842444595,01MIA0148001,ISABELLA MANILOV,ISABELLA,5,5 YOUNG PATH,YOUNG PATH,,U,...,0,0,0,0,1,female,42.294720,-71.187467,84642,NaN
47852,24445911,8424445911,05STA1949000,TINA SACHS,TINA,11,11 YOUNG PATH,YOUNG PATH,,D,...,0,0,0,0,1,female,42.294571,-71.187815,84641,NaN
47853,24445911,8424445911,07TBN1294001,BRIAN THONE,BRIAN,11,11 YOUNG PATH,YOUNG PATH,,U,...,0,0,0,0,0,male,42.294571,-71.187815,84641,NaN


In [85]:
df.loc[["RANDLETT" in x for x in df.address]]

,addressid,wp_address_id,voter_id_number,fullname,first_name,number,address,fullstname,apt,party_affiliation,...,l2_residence_addresses_addressline,l2_votertelephones_cellphoneformatted,l2_votertelephones_cellconfidencecode,l2_votertelephones_landlineformatted,l2_votertelephones_landlineconfidencecode,l2_parties_description,l2_residence_hhparties_description,l2_fullname,house_contacted,tmp
111,NaN,NaN,NaN,Maureen Grannan,NaN,66,66 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,66 Randlett Park,NaN,NaN,(617) 527-5838,2.0,Democratic,Democratic & Independent,Maureen Grannan,0,66 RANDLETT PARK
325,NaN,NaN,NaN,Laurie Hackett,NaN,61,61 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,61 Randlett Park,NaN,NaN,NaN,NaN,Democratic,Democratic & Independent,Laurie Hackett,0,61 RANDLETT PARK
630,NaN,NaN,NaN,Leah Daly,NaN,115,115 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,115 Randlett Park,NaN,NaN,(617) 244-4962,2.0,Democratic,Democratic,Leah Daly,0,115 RANDLETT PARK
899,NaN,NaN,NaN,John Ansty,NaN,121,121 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,121 Randlett Park,NaN,NaN,NaN,NaN,Democratic,Democratic,John Ansty,0,121 RANDLETT PARK
1015,NaN,NaN,NaN,Margaret Wolfe,NaN,34,34 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,34 Randlett Park,(617) 909-7036,1.0,NaN,NaN,Democratic,Democratic & Independent,Margaret Wolfe,0,34 RANDLETT PARK
1604,NaN,NaN,NaN,Charleen Kress,NaN,129,129 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,129 Randlett Park,(617) 780-0836,1.0,(617) 558-2829,2.0,Democratic,Democratic,Charleen Kress,0,129 RANDLETT PARK
1695,NaN,NaN,NaN,Edward Harrison,NaN,160,160 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,160 Randlett Park,(617) 308-7873,1.0,NaN,NaN,Democratic,Democratic & Independent,Edward Harrison,0,160 RANDLETT PARK
1722,NaN,NaN,NaN,Sarah Ansty,NaN,121,121 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,121 Randlett Park,(617) 538-4614,5.0,NaN,NaN,Democratic,Democratic,Sarah Ansty,0,121 RANDLETT PARK
1776,NaN,NaN,NaN,Tamara Wieder,NaN,135,135 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,135 Randlett Park,NaN,NaN,(617) 731-9975,2.0,Democratic,Democratic,Tamara Wieder,0,135 RANDLETT PARK
2088,NaN,NaN,NaN,Joshua Roy,NaN,79,79 RANDLETT PARK,RANDLETT PARK,NaN,NaN,...,79 Randlett Park,(781) 330-7024,5.0,NaN,NaN,Democratic,Democratic,Joshua Roy,0,79 RANDLETT PARK


In [61]:
df.columns

Index(['addressid', 'wp_address_id', 'voter_id_number', 'fullname',
       'first_name', 'number', 'address', 'fullstname', 'apt',
       'party_affiliation', 'age_on_election_day', 'ward', 'precinct',
       'occupation', 'nationality', 'party_name', 'gender', 'lat', 'lon',
       'form_id', 'canvass_list', 'l2_voters_statevoterid',
       'l2_voters_sequenceoddeven', 'l2_voters_age', 'l2_ethnic_description',
       'l2_voterparties_change_changed_party', 'l2_vote_frequency',
       'l2_votertelephones_cellphoneformatted',
       'l2_votertelephones_cellconfidencecode',
       'l2_votertelephones_landlineformatted',
       'l2_votertelephones_landlineconfidencecode', 'l2_parties_description',
       'l2_residence_hhparties_description', 'l2_fullname', 'house_contacted'],
      dtype='object')

In [9]:
l2_df

,LALVOTERID,Voters_Active,Voters_StateVoterID,Voters_CountyVoterID,Voters_FirstName,Voters_MiddleName,Voters_LastName,Voters_NameSuffix,VoterTelephones_LandlineFormatted,VoterTelephones_LandlineUnformatted,...,Local_or_Municipal_2022_10_03,Local_or_Municipal_2022_09_13,Local_or_Municipal_2022_09_06,Local_or_Municipal_2022_06_28,Local_or_Municipal_2022_06_21,Local_or_Municipal_2022_06_14,Local_or_Municipal_2022_06_13,Local_or_Municipal_2022_06_07,Local_or_Municipal_2022_06_06,Local_or_Municipal_2022_06_04
0,LALMA159116231,Active,05FSN0550001,NaN,Susan,Helen,Feldman,NaN,(617) 244-7484,6.172447e+09,...,0,0,0,0,0,0,0,0,0,0
1,LALMA159116696,Active,01MRT0250001,NaN,Robert,Anthony,Monico,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,LALMA159119629,Active,01WDE0850000,NaN,Denise,Madeleine,Wernikoff,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,LALMA159121891,Active,01SLA0551000,NaN,Lisa,Berlin,Sazer,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,LALMA159122084,Active,01CFD0651000,NaN,Fred,L,Chanowski,NaN,(617) 965-2623,6.179653e+09,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,LALMA162769061,Active,01BNS0180005,NaN,Nicholas,Doran,Brandt,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8134,LALMA162773447,Active,05ZJA2283000,NaN,Jessica,Ann,Zerillo,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8135,LALMA552645533,Active,02OEN0356003,NaN,Eileen,B,Okeefe,NaN,(617) 964-4611,6.179645e+09,...,0,0,0,0,0,0,0,0,0,0
8136,LALMA552639089,Active,05ZYG1587000,NaN,Yangyang,NaN,Zhao,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [26]:
voters_df.sort_values(by=['ward','precinct','fullstname'],ascending=True,inplace=True)


addresses_df=voters_df[['ward','precinct','addressid','wp_address_id']].groupby(['ward','precinct','addressid']).first().reset_index()
addresses_df['form_id']=addresses_df[['ward','precinct','addressid','wp_address_id']].groupby(['ward','precinct']).cumcount()
addresses_df['form_id']=[f'{w}{p}{f}' for w,p,f in zip(addresses_df.ward,addresses_df.precinct,addresses_df.form_id)]
addresses_df
voters_df=voters_df.merge(addresses_df[['form_id','addressid']],on='addressid')
voters_df.to_csv('~/Desktop/voters_with_formid.csv')

In [27]:
df=pd.read_csv('~/Desktop/selected_voters_1_1.csv')
df.fillna(' ',inplace=True)


In [47]:
config = {
    "write_out_cols": [
        "form_id",
        "address",
        "apt",
        "fullname",
        "age_on_election_day",
        "occupation",
    ],
    "groupby_cols": ["form_id", "address", "apt", "fullname"],
    "roads": {
        "6A1": ["BURR RD", "ELLISON RD", "FENNO RD", "MANAMET RD"],
        "6B1": ["MORELAND AVE", "ALBION PL", "NEWBURY ST", "KENMORE ST"],
    },
}
from utils.io import dict_to_yaml, yaml_to_dict

dict_to_yaml(config, "canvass.yml")

In [48]:
d = yaml_to_dict("/Users/lindseygulden/dev/newton/voters/canvass.yml")

In [56]:
roadlist = ["NEHOLDEN RD", "FISHER AVE", {"WALNUT ST": [960, 1080]}]

r = [list(x.values())[0] if isinstance(x, dict) else x for x in roadlist]
r

['NEHOLDEN RD', 'FISHER AVE', [960, 1080]]

In [ ]:
for x in roadlist:
    if isinstance(x,dict):
        streetname=list(x.keys())[0]
        numbers=list(x.values())[0]
        out_list.append(out_df.loc[(out_df.fullstname==streetname) & (out_df.number>=numbers[0] & out_df.number<=numbers[1])])
    else:
        out_list.append(out_df.loc[(out_df.fullstname==x)])

{'voter_file': '/Users/lindseygulden/Desktop/voters_with_formid.csv',
 'output_dir': '/Users/lindseygulden/Desktop/canvass_lists',
 'list_generation_id': 'canvass_set_1',
 'groupby_cols': ['form_id', 'address', 'apt', 'fullname'],
 'street_col': 'fullstname',
 'roads': {'6A1': ['BURR RD', 'ELLISON RD', 'FENNO RD', 'MANEMET RD'],
  '6B1': ['KENMORE ST',
   'PINE CREST RD',
   'MORELAND AVE',
   'ALBION PL',
   'NEWBURY ST'],
  '6B2': ['KINGMAN RD', 'FISHER AVE', {'WALNUT ST': [960, 1080]}],
  '6B3': ['SAXON RD', 'LAKEWOOD RD', 'NORMAN RD'],
  '6B4': ['HYDE AVE']},
 'write_out_cols': ['form_id',
  'address',
  'apt',
  'fullname',
  'age_on_election_day',
  'occupation']}

In [58]:
sorted(list(voters_df.fullstname.unique()))

['ABBOTT ST',
 'ABERDEEN ST',
 'ACACIA AVE',
 'ACADEMY RD',
 'ACORN DR',
 'ADAMS AVE',
 'ADAMS CT',
 'ADAMS ST',
 'ADAMS TER',
 'ADELINE RD',
 'ADELLA AVE',
 'ADENA RD',
 'AGAWAM RD',
 'ALBA CIR',
 'ALBAN RD',
 'ALBEMARLE RD',
 'ALBERT RD',
 'ALBION PL',
 'ALBION ST',
 'ALDEN PL',
 'ALDEN ST',
 'ALDERWOOD RD',
 'ALEXANDER RD',
 'ALGONQUIN RD',
 'ALLEN AVE',
 'ALLERTON RD',
 'ALLISON ST',
 'ALLSTON ST',
 'AMHERST RD',
 'AMY CIR',
 'ANDREW ST',
 'ANGIER CIR',
 'ANITA CIR',
 'ANNAPOLIS RD',
 'ANNAWAN RD',
 'ANTHONY CIR',
 'ANTHONY RD',
 'ANTONELLIS CIR',
 'APPLEGARTH ST',
 'APPLETON CIR',
 'ARAPAHOE RD',
 'ARBOR RD',
 'ARDEN RD',
 'ARDMORE RD',
 'ARDMORE TER',
 'ARLINGTON ST',
 'ARLO RD',
 'ARNOLD RD',
 'ARUNDEL TER',
 'ASCENTA TER',
 'ASH ST',
 'ASHCROFT RD',
 'ASHEVILLE RD',
 'ASHFORD RD',
 'ASHMONT AVE',
 'ASHMONT RD',
 'ASHTON AVE',
 'ASPEN AVE',
 'ATHELSTANE RD',
 'ATKINSON ST',
 'ATWOOD AVE',
 'AUBURN ST',
 'AUBURN TER',
 'AUBURNDALE AVE',
 'AUDUBON DR',
 'AUSTIN ST',
 'AVALON RD',


In [57]:
voters_df.loc[voters_df.fullstname == "NEHOLDEN RD"]

,addressid,wp_address_id,voter_id_number,fullname,first_name,number,address,fullstname,apt,party_affiliation,...,finance,muckity_muck,helping,multifamily,men_over_60,women,gender,lat,lon,form_id


In [44]:
for name, roadlist in config["roads"].items():
    print(
        df[config["write_out_cols"]]
        .loc[df.fullstname.isin(roadlist)]
        .groupby(config["groupby_cols"])
        .first()
    )

Empty DataFrame
Columns: [age_on_election_day, occupation]
Index: []
                                                   age_on_election_day  \
form_id address         apt fullname                                     
640     3 ALBION PL         JENNA CASTELLOT                         35   
                            JOHN CASTELLOT                          73   
                            KATHRYN BLUM                            74   
                            MICHAEL BLUM                            72   
                            VIRGINIA MERLINI                        73   
6321    12 ALBION PL        MARCIA ROSENBAUM                        67   
6322    16 ALBION PL        JARED KESSELHEIM                        48   
6323    2 ALBION PL         ADELE KAUFFMAN                          72   
                            BARRY COHEN                             73   
6325    25 ALBION PL        BARBARA ATWOOD                          84   
                            EBEN ATWOOD    

In [46]:
sorted(voters_df.fullstname.unique())

['ABBOTT ST',
 'ABERDEEN ST',
 'ACACIA AVE',
 'ACADEMY RD',
 'ACORN DR',
 'ADAMS AVE',
 'ADAMS CT',
 'ADAMS ST',
 'ADAMS TER',
 'ADELINE RD',
 'ADELLA AVE',
 'ADENA RD',
 'AGAWAM RD',
 'ALBA CIR',
 'ALBAN RD',
 'ALBEMARLE RD',
 'ALBERT RD',
 'ALBION PL',
 'ALBION ST',
 'ALDEN PL',
 'ALDEN ST',
 'ALDERWOOD RD',
 'ALEXANDER RD',
 'ALGONQUIN RD',
 'ALLEN AVE',
 'ALLERTON RD',
 'ALLISON ST',
 'ALLSTON ST',
 'AMHERST RD',
 'AMY CIR',
 'ANDREW ST',
 'ANGIER CIR',
 'ANITA CIR',
 'ANNAPOLIS RD',
 'ANNAWAN RD',
 'ANTHONY CIR',
 'ANTHONY RD',
 'ANTONELLIS CIR',
 'APPLEGARTH ST',
 'APPLETON CIR',
 'ARAPAHOE RD',
 'ARBOR RD',
 'ARDEN RD',
 'ARDMORE RD',
 'ARDMORE TER',
 'ARLINGTON ST',
 'ARLO RD',
 'ARNOLD RD',
 'ARUNDEL TER',
 'ASCENTA TER',
 'ASH ST',
 'ASHCROFT RD',
 'ASHEVILLE RD',
 'ASHFORD RD',
 'ASHMONT AVE',
 'ASHMONT RD',
 'ASHTON AVE',
 'ASPEN AVE',
 'ATHELSTANE RD',
 'ATKINSON ST',
 'ATWOOD AVE',
 'AUBURN ST',
 'AUBURN TER',
 'AUBURNDALE AVE',
 'AUDUBON DR',
 'AUSTIN ST',
 'AVALON RD',
